# Analysis

In this analysis, unit of time used in simulation is minutes.

Credit:

* Analysis of the warm-up period, number of replications and spread of results across replications is adapted from Tom Monks (2024) HPDM097 - Making a difference with health data (https://github.com/health-data-science-OR/stochastic_systems) (MIT License). 

License:

* This project is licensed under the MIT License. See the LICENSE file for more details.

TODO: Create Latex-formatted table.

TODO: Mention alternative methods for choosing warm-up period length (or keep it simple?)

TODO: Mention alternative methods for choosing number of replications (or keep it simple?)

TODO: Consider whether to add time unit to model docstrings etc.

In [1]:
# Model code
from model import Defaults, Trial

# Other dependencies
import numpy as np
import time
import itertools
from IPython.display import display
import pandas as pd
import plotly.express as px
import scipy.stats as st

## Choosing length of warm-up period

A suitable length for the warm-up period can be determined using the **time series inspection approach**. This involves looking at performance measures over time to identify when the system is exhibiting **steady state behaviour** (even though the system will never truly reach a "steady state").

If we simply plot the mean result at regular intervals, this would vary too much. Therefore, we plot the **cumulative mean** of the performance measure, and look for the point at which this **smoothes out and stabilises**. This indicates the point for the warm-up period to end.

This should be assessed when running the model using the base case parameters. If these change, you should reassess the appropriate warm-up period.

In [2]:
def time_series_inspection(data_collection_period, warm_up=None):
    """
    Time series inspection method for determining length of warm-up.

    Arguments:
        data_collection_period (float):
            Length of time to run the simulation for.
        warm_up (float, optional):
            Location on X axis to plot vertical red line indicating the chosen
            warm-up period. Defaults to None, which will not plot a line.
    """
    # Use default parameters, but with no warm-up and specified run length
    param = Defaults()
    param.warm_up_period = 0
    param.data_collection_period = data_collection_period
    # display(param.__dict__)

    # Run model
    choose_warmup = Trial(param)
    choose_warmup.run_trial()

    # Filter to nurse results
    nurse = choose_warmup.interval_audit_df[
        choose_warmup.interval_audit_df['resource_name'] == 'nurse']

    # Define columns to analyse
    plot = {
        'utilisation': 'Cumulative mean nurse utilisation',
        'running_mean_wait_time': 'Running mean nurse wait time'
    }
    for var, label in plot.items():
        # Reformat so index is simulation time and columns are each run
        reformat = (
            nurse[['simulation_time', var, 'run']]
            .pivot(index='simulation_time',
                columns='run',
                values=var))

        # Find mean at each time interval across replications
        # Then, for utilisation, calculate cumulative mean (not necessary
        # for wait time as that is already a running mean)
        if var == 'utilisation':
            cumulative = reformat.mean(axis=1).expanding().mean()
        elif var == 'running_mean_wait_time':
            cumulative = reformat.mean(axis=1)

        # Create plot. If specified, add vertical line to indicate suggested
        # warm-up length
        fig = px.line(cumulative)
        fig.update_layout(
            xaxis_title = 'Run time (minutes)',
            yaxis_title = label,
            showlegend=False
        )
        if warm_up is not None:
            fig.add_vline(x=warm_up, line_color='red', line_dash='dash')
        fig.show()

Having run the model for three days, it appears to reach a steady state at around 2500 minutes.

In [3]:
time_series_inspection(data_collection_period=1440*3, warm_up=2520)

However, it is important to look far ahead - so we run it for more days, and find actually a later warm-up is more appropriate.

In [4]:
time_series_inspection(data_collection_period=1440*40, warm_up=1440*13)

## Choosing the number of replications

The **confidence interval method** can be used to select the number of replications to run. The more replications you run, the narrower your confidence interval becomes, leading to a more precise estimate of the model's mean performance.

First, you select a desired confidence interval - for example, 95%. Then, run the model with an increasing number of replications, and identify the number required to achieve that precision in the estimate of a given metric - and also, to maintain that precision (as the intervals may converge or expand again later on).

This method is less useful for values very close to zero - so, for example, when using utilisation (which ranges from 0 to 1) it is recommended to multiple values by 100.

In [5]:
# TODO

## Spread of replication results

In [6]:
# Set model parameters
param = Defaults()
param.number_of_runs = 50
display(param.__dict__)

# Run trial
trial = Trial(param)
trial.run_trial()

{'_initialising': False,
 'patient_inter': 4,
 'mean_n_consult_time': 10,
 'number_of_nurses': 5,
 'warm_up_period': 18720,
 'data_collection_period': 43200,
 'number_of_runs': 50,
 'audit_interval': 120,
 'scenario_name': 0}

In [7]:
def plot_results_spread(column, x_label, y_label='Frequency'):
    """
    Plot spread of results from across replications, for chosen column.

    Arguments:
        column (str):
            Name of column to plot.
        x_label (str):
            X axis label.
        y_label (str):
            Y axis label
    """
    fig = px.histogram(trial.trial_results_df[column])
    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label
    )
    fig.show()


plot_results_spread('arrivals', 'Arrivals')
plot_results_spread('mean_q_time_nurse', 'Mean wait time for nurse')
plot_results_spread('mean_time_with_nurse', 'Mean length of nurse consultation')
plot_results_spread('average_nurse_utilisation', 'Mean nurse utilisation')

## Other stuff

In [8]:
single_trial = Trial(param=Defaults())
single_trial.run_trial()
single_trial.trial_results_df

uncertainty_metrics = {}
for col in ['mean_q_time_nurse', 'average_nurse_utilisation']:
    # Calculate mean, standard deviation and count
    data = single_trial.trial_results_df[col]
    mean = data.mean()
    std_dev = data.std()
    n = len(data)

    # Calculate 95% confidence interval
    ci_lower, ci_upper = st.t.interval(
        confidence=0.95, df=len(data)-1, loc=mean, scale=st.sem(data))

    # Save as dictionary
    uncertainty_metrics[col] = {
        'mean': mean,
        'std_dev': std_dev,
        'lower_95_ci': ci_lower,
        'upper_95_ci': ci_upper
    }
# Convert to dataframe
pd.DataFrame(uncertainty_metrics)

,mean_q_time_nurse,average_nurse_utilisation
mean,0.499556,0.500332
std_dev,0.064223,0.004349
lower_95_ci,0.453613,0.497221
upper_95_ci,0.545498,0.503443


In [10]:
# Run a single trial
single_trial = Trial(param=Defaults())
single_trial.run_trial()

# Preview results
display(single_trial.patient_results_df)
display(single_trial.trial_results_df)
display(single_trial.interval_audit_df)

# Plot interval audit utilisation
fig = px.line(single_trial.interval_audit_df,
              x='simulation_time', y='utilisation', color='run')
fig.show()

# Calculate and plot median utilisation
interval_audits_median = (single_trial
                          .interval_audit_df
                          .drop('resource_name', axis=1)
                          .groupby('simulation_time')
                          .median()
                          .reset_index())
fig = px.line(interval_audits_median,
              x='simulation_time',
              y='utilisation')
fig.show()


,patient_id,arrival_time,q_time_nurse,time_with_nurse,run
0,1,18720.556711,0.0,1.551695,0
1,2,18724.792565,0.0,3.649895,0
2,3,18725.627670,0.0,8.137105,0
3,4,18728.774983,0.0,22.057543,0
4,5,18729.812428,0.0,15.439297,0
...,...,...,...,...,...
108017,10701,61913.932787,0.0,11.338667,9
108018,10702,61915.218078,0.0,38.058422,9
108019,10703,61918.140880,0.0,20.107511,9
108020,10704,61918.520147,0.0,8.697632,9


,run_number,scenario,arrivals,mean_q_time_nurse,mean_time_with_nurse,average_nurse_utilisation
0,0,0,10972,0.504541,9.842268,0.499639
1,1,0,10784,0.514151,10.060481,0.501991
2,2,0,10854,0.523235,9.925025,0.498130
3,3,0,10831,0.479149,9.937057,0.498220
4,4,0,10720,0.461457,10.015904,0.496870
5,5,0,10772,0.388265,9.884996,0.492904
6,6,0,10831,0.466938,10.041800,0.503356
7,7,0,10781,0.625888,10.086979,0.503449
8,8,0,10772,0.468497,10.202270,0.508791
9,9,0,10705,0.563435,10.092602,0.499972


,resource_name,simulation_time,utilisation,queue_length,running_mean_wait_time,run
0,nurse,18720,0.6,0,0.427625,0
1,nurse,18840,0.4,0,0.425181,0
2,nurse,18960,0.6,0,0.423041,0
3,nurse,19080,1.0,3,0.427408,0
4,nurse,19200,0.6,0,0.440854,0
...,...,...,...,...,...,...
3595,nurse,61320,0.4,0,0.524115,9
3596,nurse,61440,0.6,0,0.523119,9
3597,nurse,61560,0.8,0,0.522416,9
3598,nurse,61680,0.2,0,0.521299,9


In [11]:
# Run with 1 to 14 cores
speed = []
param=Defaults()
param.number_of_runs = 100
for i in range(1, 15, 1):
    start_time = time.time()
    my_trial = Trial(param)
    my_trial.run_trial(cores=i)
    run_time = round((time.time() - start_time), 3)
    speed.append({'Cores': i, 'Run Time (seconds)': run_time})

# Display and plot time by number of cores
timing_results = pd.DataFrame(speed)
print(timing_results)
fig = px.line(timing_results, x='Cores', y='Run Time (seconds)')
fig.show()

    Cores  Run Time (seconds)
0       1              12.085
1       2               6.442
2       3               4.790
3       4               3.784
4       5               3.293
5       6               3.140
6       7               3.047
7       8               2.465
8       9               2.327
9      10               2.293
10     11               2.711
11     12               2.142
12     13               2.034
13     14               1.973


In [12]:
# TODO: Alter how this runs so fresh Defaults() each time

# Define a set of scenarios
param = Defaults()
param.number_of_runs = 5
scenarios = {
    'patient_inter': [5, 10, 15],
    'mean_n_consult_time': [15, 20, 35],
    'number_of_nurses': [3, 6, 9]
}

# Find every possible permutation of the scenarios
all_scenarios_tuples = list(itertools.product(*scenarios.values()))
# Convert back into dictionaries
all_scenarios_dicts = [
    dict(zip(scenarios.keys(), p)) for p in all_scenarios_tuples]
# Preview some of the scenarios
print(f'There are {len(all_scenarios_dicts)} scenarios. For example:')
display(all_scenarios_dicts[0:6])

# Run the scenarios...
results = []
for index, scenario_to_run in enumerate(all_scenarios_dicts):
    # Overwrite defaults from the passed dictionary
    param.scenario_name = index
    for key in scenario_to_run:
        setattr(param, key, scenario_to_run[key])
    # Run trial and keep trial-level results
    my_trial = Trial(param)
    my_trial.run_trial()
    results.append(my_trial.trial_results_df)
# View mean results by scenario
display(pd.concat(results)
        .drop('run_number', axis=1)
        .groupby('scenario')
        .mean()
        .head(20))

# TODO: Issue: warm-up patients use resources but their activity is excluded
# from metrics. Post-warm-up patients queue behind these, making it look
# like resources are under-utilised during the measurement period if there are
# long queues (e.g. due to really short inter-arrival times)

There are 27 scenarios. For example:


[{'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 3},
 {'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 6},
 {'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 9},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 3},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 6},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 9}]

,arrivals,mean_q_time_nurse,mean_time_with_nurse,average_nurse_utilisation
scenario,,,,
0,8675.0,438.223177,14.973316,0.997347
1,8675.0,0.499985,14.973353,0.500730
2,8675.0,0.014008,14.973171,0.333867
3,8675.0,11071.520375,19.900799,0.999506
4,8675.0,2.986440,19.963676,0.667406
5,8675.0,0.101103,19.963760,0.445021
6,8675.0,30438.035146,34.722447,0.999212
7,8675.0,6177.665586,34.798605,0.999169
8,8675.0,7.109108,34.934277,0.778212
